   ### ✨ Analytical Project ✨ | E - COMMERCE |
***
   #### owner: Golovin Alexey
   
***

<b>Project Task</b>

1. How many users have made a single purchase?

2. On average, how many orders are not delivered due to various reasons (Make a detailed breakdown of reasons)? 

3. For each product, determine the day of the week on which the product is most frequently bought

4. What is the average number of purchases per customer per week (in month)? Please note that within a month, the number of weeks may not be a whole number. For example, November 2021 had 4.28 weeks

5. Using pandas, perform a cohort analysis of customers for the period from January to December and identify the cohort(s) with the highest retention for the 3rd month.

6. "Frequently, for high-quality analysis of audience, segmentation-based methods are used. Use Python to perform RFM segmentation of customers in order to make a high-quality evaluation of the audience. For clustering, you can use the following metrics: R (Recency) - time elapsed since the last purchase before the current date, F (Frequency) - total number of purchases made over time, and M (Monetary) - total amount spent over time.

<b>olist_customers_datase.csv</b> — dataset of unique customer <br>
<i>customer_id </i> — id of customer for each order <br> 
<i>customer_unique_id </i> —  unique id of customer <br>
<i>customer_zip_code_prefix </i>—  zip-code  <br>
<i>customer_city </i> —  city of delivery <br>
<i>customer_state</i> —  state of delivery <br>

<b>olist_orders_dataset.csv </b> —  orders dataset <br>
<i>order_id</i> —  unique order id (check) <br>
<i>customer_id</i> —  id of customer for each order <br>
<i>order_status</i> —  order status <br>
<i>order_purchase_timestamp</i> —  time/date of purchase <br>
<i>order_approved_at</i> —  time/date of approving order <br>
<i>order_delivered_carrier_date</i> — time/date of issuance for logistic-partner <br>
<i>order_delivered_customer_date</i> — time/date of delivery <br>
<i>order_estimated_delivery_date</i> — estimated_delivery time/date  <br>

<b>olist_order_items_dataset.csv</b> —  items in order <br>
<i>order_id</i> —  unique order id (check) <br>
<i>order_item_id</i> —  id inside one order <br>
<i>product_id</i> —  product id (barcode) <br>
<i>seller_id</i> — id of distributor <br>
<i>shipping_limit_date</i> —  maximum date for delivery, before order will be giving to logistic-partner <br>
<i>price</i> —  price per unit <br>
<i>freight_value </i> — weight of product <br>


Example of structure of data, you can visualise on <b>order_id</b> == `00143d0f86d6fbd9f9b38ab440ac16f5`  <br>


Unique statuses for orders in dataset *olist_orders_dataset*: <br>

* created  <br>
* approved  <br>
* invoiced  <br>
* processing <br>
* shipped <br>
* delivered <br>
* unavailable <br>
* canceled<br>


At first, I will add a block with library imports, then I will load the datasets and perform exploratory data analysis (EDA)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

I will read the datasets and assign names, and while reading the **'orders'** and **'items'** datasets, I will change the date format to **'datetime'**.

In [2]:
url_customers = './olist_customers_dataset.csv'
url_items = './olist_order_items_dataset.csv'
url_orders = './olist_orders_dataset.csv'

customers = pd.read_csv(url_customers)
items = pd.read_csv(url_items, parse_dates=['shipping_limit_date'])
orders = pd.read_csv(url_orders, parse_dates=['order_purchase_timestamp', 
                                              'order_approved_at',
                                              'order_delivered_carrier_date',
                                              'order_delivered_customer_date',
                                              'order_estimated_delivery_date']
                    )

Let's delve into our datasets and check for missing values or interesting insights upon first glance

In [11]:
customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [4]:
customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [5]:
customers.isna().sum()   #using .isna to check number of missing values in columns

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [7]:
customers.nunique()   # this method return number of unique values

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

As we see: total value of customers = **96 096**

In [10]:
items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [13]:
items.describe()   # The describe() function displays descriptive statistics for numerical columns"

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [14]:
items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [15]:
items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [16]:
items.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [17]:
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [18]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [19]:
orders.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

In [20]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [21]:
pd.unique(orders['order_status']) # check unique status of orders

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [22]:
orders.order_status.unique()  # second way to check unique values in columns

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

### 1. How many users have made a single purchase?
***

Merge customers dataframe with orders, and assign variable:

In [23]:
customers_and_orders = customers.merge(orders, how = 'inner', on = 'customer_id') 
customers_and_orders.head(3)

# with method .merge, in parameters choose type of join and column or index level names to join on

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13


It's reasonable decision to consider only that 'purchase' will be the orders with status "delivered", and have a date in column order_delivered_customer_date. As a full cycle of deal.

In [24]:
customers_and_orders[['customer_unique_id','order_status', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']] \
                    .groupby('order_status',as_index = False) \
                    .count()

,order_status,customer_unique_id,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date
0,approved,2,2,0,0
1,canceled,625,484,75,6
2,created,5,0,0,0
3,delivered,96478,96464,96476,96470
4,invoiced,314,314,0,0
5,processing,301,301,0,0
6,shipped,1107,1107,1107,0
7,unavailable,609,609,0,0


In [26]:
buy = customers_and_orders \
                    .fillna(0) \
                    .query('order_status == "delivered"') \
                    .query('order_delivered_customer_date != 0') 
# filtering orders with status "delivered" and droping orders with empty date of delivery

Consider the number of buyers with orders equal to 1

In [27]:
buy.groupby('customer_unique_id', as_index = False) \
   .agg({'order_status': 'count'}) \
   .query('order_status == 1') \
   .nunique()

customer_unique_id    90549
order_status              1
dtype: int64

Answer - **90 549** customers, who made a single purchase.